In [1]:
import os
import sys
import fitsio
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import matplotlib as mpl

from   astropy.table import Table

In [2]:
cwd = os.getcwd()
code_root = '/'.join(x for x in cwd.split('/')[:-2])

sys.path.append(code_root)

print(cwd)
print(code_root)

/cosma/home/durham/dc-moor2/DESI/docs/nb
/cosma/home/durham/dc-moor2/DESI


In [3]:
import pubplot

from   cosmo         import cosmo, fsky
from   cartesian     import rotate
from   findfile      import findfile
# from   poisson     import fillfactor_expectation, fillfactor_poisson

In [4]:
survey  = 'gama'
field   = 'G9'
dryrun  = False
realz   = 0
version = 'GAMA4'

In [5]:
nbar   = 2.
radius = 8.

In [6]:
fpath = findfile(ftype='randoms', field=field, dryrun=dryrun, survey=survey, version=version)
rand  = Table.read(fpath)
rand.meta

OrderedDict([('ZMIN', 0.039),
             ('ZMAX', 0.263),
             ('DZ', 0.0001),
             ('NRAND', 2531909),
             ('FIELD', 'G9'),
             ('AREA', 60.0),
             ('VOL', 2531908.950723488),
             ('RAND_DENS', 1.000000019462197),
             ('VOL8', 2144.660584850632),
             ('NRAND8', 2144.66062659044),
             ('NRAND8_PERR', 46.31048074238098),
             ('EXTNAME', 'RANDOMS')])

In [7]:
bound = Table.read(fpath, 'BOUNDARY')
bound

BOUND_RA,BOUND_DEC,Z,V,BOUNDID,FIELD,GAMA_FIELD,CARTESIAN_X,CARTESIAN_Y,CARTESIAN_Z,ROTCARTESIAN_X,ROTCARTESIAN_Y,ROTCARTESIAN_Z
float64,float64,float64,float64,int64,bytes2,bytes3,float64,float64,float64,float64,float64,float64
141.0,0.40063778486427637,0.07144800497783309,47927.407498281864,0,G9,G9,-164.1828944946481,132.95268648550646,1.4772760780365697,210.11147553008232,22.083091861130512,-0.3562827379473492
141.0,0.9719219102434473,0.04367667418526872,3816.802902615331,1,G9,G9,-100.90069403813077,81.70777097037293,2.2026320706884337,129.13798862230786,13.571446052003303,1.0757457898906768
129.0,2.7601640018222424,0.12577607528972545,293793.99550875596,2,G9,G9,-231.24461942800903,285.56332299567856,17.715288512555983,365.5790789783938,-38.40912363796099,14.525602792833633
129.0,1.31087012077912,0.08313844598273315,80366.67149343388,3,G9,G9,-154.31199649272307,190.55944568811063,5.611005632628955,243.90061456629806,-25.630817126773366,3.4827308732609294
141.0,2.6954966166290735,0.18725231391459907,953718.15460282,4,G9,G9,-419.7640439059028,339.91822046436795,25.42959051585407,537.3779388733319,56.45952320493319,20.74093260739289
129.0,1.9719885776508628,0.17165427393637708,739844.1111491593,5,G9,G9,-312.788763146469,386.2619542056905,17.113255489622375,494.4340205018058,-51.953391633390254,12.79904680659359
129.0,1.7307081189790998,0.08372085245391617,82237.1298696577,6,G9,G9,-155.3447619029263,191.83480475653835,7.458613884631363,245.54876611095474,-25.802356747576752,5.316026278573695
129.0,-0.09175657507305623,0.2463790153223925,2102735.415932571,7,G9,G9,-442.1049196788346,545.9541082015504,-1.1250413304109104,698.6268181583906,-73.43246542513236,-7.221908047315347
141.0,-0.5835953732819252,0.1403536594851054,408189.51037352736,8,G9,G9,-318.0613922883886,257.56103705091067,-4.168814669278524,406.9746455568911,42.78021145261579,-7.7205873369774345


In [8]:
fpath = findfile(ftype='randoms_n8', field=field, dryrun=dryrun, survey=survey, version=version)
rand  = Table.read(fpath)
rand.meta

OrderedDict([('ZMIN', 0.039),
             ('ZMAX', 0.263),
             ('DZ', 0.0001),
             ('NRAND', 2531909),
             ('FIELD', 'G9'),
             ('AREA', 60.0),
             ('VOL', 2531908.950723488),
             ('RAND_DENS', 1.000000019462197),
             ('VOL8', 2144.660584850632),
             ('NRAND8', 2144.66062659044),
             ('NRAND8_PERR', 46.31048074238098),
             ('EXTNAME', 'RANDOMS'),
             ('RSPHERE', 8.0)])

In [9]:
fpath = findfile(ftype='randoms_bd', field=field, dryrun=dryrun, survey=survey, version=version)
rand  = Table.read(fpath)
rand.meta

OrderedDict([('ZMIN', 0.039),
             ('ZMAX', 0.263),
             ('DZ', 0.0001),
             ('NRAND', 2531909),
             ('FIELD', 'G9'),
             ('AREA', 60.0),
             ('VOL', 2531908.950723488),
             ('RAND_DENS', 1.000000019462197),
             ('VOL8', 2144.660584850632),
             ('NRAND8', 2144.66062659044),
             ('NRAND8_PERR', 46.31048074238098),
             ('EXTNAME', 'RANDOMS'),
             ('RSPHERE', 8.0)])

In [10]:
rand

RANDOM_RA,RANDOM_DEC,Z,V,RANDID,FIELD,GAMA_FIELD,CARTESIAN_X,CARTESIAN_Y,CARTESIAN_Z,ROTCARTESIAN_X,ROTCARTESIAN_Y,ROTCARTESIAN_Z,RAND_N8,FILLFACTOR,BOUND_DIST,BOUNDID,FILLFACTOR_POISSON
float64,float64,float32,float32,int64,bytes2,bytes3,float32,float32,float32,float64,float64,float64,int32,float64,float64,int64,float64
133.34661491292866,1.929788236569844,0.16637072,684545.6,752457,G9,G9,-331.03702,350.71582,16.249653,482.19695562670597,-13.871838421403567,12.033830953986604,2205,1.0,9.025652605385531,467854,1.0281346953739188
130.8820823131463,-0.01855369585325641,0.26052988,2474504.2,2261570,G9,G9,-484.7082,559.91614,-0.2398146,738.6353715115721,-53.113891664440985,-6.698624475655291,2064,0.9623900277785797,6.622303961265657,429513,0.9623900277785797
132.98722987268664,1.8605792121910472,0.2547674,2322668.5,707931,G9,G9,-494.14148,530.1387,23.542362,724.4558816257608,-25.38898986851109,17.208450307614598,2065,1.0,14.41934464669521,385342,0.9628563020168446
138.12042429211397,-0.5355639117771467,0.18362801,910473.3,305178,G9,G9,-395.09552,354.24478,-4.9603214,529.7973909540908,28.933290714753298,-9.59318355094465,2164,1.0,13.562981804876634,467451,1.0090174516050614
137.06820930281762,2.5310249947192767,0.19239321,1041370.7,2456335,G9,G9,-405.93704,377.63992,24.507898,554.2639047031877,20.05870960314985,19.662221851217378,1902,0.8868536011796796,4.6558707062791855,398219,0.8868536011796796
132.30231431294314,0.7796800537063722,0.07972012,79413.23,117823,G9,G9,-158.38162,174.04501,3.2024546,235.0811352595224,-11.0546370582434,1.1469724828245647,2191,1.0,9.121479926654601,365721,1.0216068560382114
131.7145215508942,2.479245873862223,0.20771444,1297747.9,964875,G9,G9,-397.03067,445.39014,25.834295,595.8876457274478,-34.1419550961171,20.624701759786472,1990,0.9278857341469834,5.48305935821094,484811,0.9278857341469834
132.56903216738564,-0.7052120842979612,0.1538359,545439.56,223791,G9,G9,-302.60403,329.4366,-5.506049,446.8566971792397,-18.933471721273293,-9.413679288813377,2159,1.0,10.116197197413161,538841,1.0066860804137372
140.18830970953306,1.353638835044582,0.12571643,302896.8,1926525,G9,G9,-282.3773,235.36551,8.686464,366.15842270255314,33.27513740886951,5.485021373590347,1937,0.9031731995189481,5.221284278906502,378834,0.9031731995189481


In [11]:
fpath = findfile(ftype='randoms_bd_ddp_n8', field=field, dryrun=dryrun, survey=survey, version=version)

rand  = Table.read(fpath)
rand.meta

FileNotFoundError: [Errno 2] No such file or directory: '/cosma/home/durham/dc-moor2/data/GAMA4/randoms/randoms_bd_ddp_n8_G9_0.fits'

In [ ]:
pl.plot(rand['RANDOM_RA'], rand['RANDOM_DEC'], marker=',', lw=0.0, c='k')
pl.plot(bound['BOUND_RA'], bound['BOUND_DEC'], marker=',', lw=0.0, c='cyan')
pl.xlabel('RA [deg.]')
pl.ylabel('DEC [deg.]')

In [ ]:
_ = pl.hist(rand['V'], bins=100, histtype='step', color='k')
_ = pl.hist(bound['V'], bins=100, histtype='step', color='c')

pl.xlabel('V')
pl.yscale('log')

In [ ]:
_ = pl.hist(rand['Z'], bins=100, histtype='step', color='k')

pl.axvline(300. / 2.9979e5, lw=0.5, c='k')

pl.yscale('log')
pl.xlabel('$z$')

pl.xlim(0.0, 0.6)

In [ ]:
pos       = np.c_[rand['ROTCARTESIAN_X'], rand['ROTCARTESIAN_Y'], rand['ROTCARTESIAN_Z']]
pos_bound = np.c_[bound['ROTCARTESIAN_X'], bound['ROTCARTESIAN_Y'], bound['ROTCARTESIAN_Z']]

In [ ]:
pl.plot(pos[::1,0], pos[::1,1], marker=',', lw=0.0, c='k')

pl.xlabel('$x$')
pl.ylabel('$y$')

In [ ]:
pl.plot(pos_bound[:,0], pos_bound[:,1], marker=',', lw=0.0, c='k')
pl.xlabel('$x$')
pl.ylabel('$y$')

In [ ]:
fig = plt.figure(figsize=(10,10))
ax  = fig.add_subplot(projection='3d')

ax.scatter(pos[::1, 0],\
           pos[::1, 1],\
           pos[::1, 2],\
           marker='.',\
           lw=0.0)

ax.set_xlim(0.0, 1000.)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax  = fig.add_subplot(projection='3d')

ax.scatter(pos_bound[:,0][::10],\
           pos_bound[:,1][::10],\
           pos_bound[:,2][::10],\
           marker='.',\
           lw=0.0)

ax.set_xlim(0.0, 1000.)

In [ ]:
np.mean(rand['V'] < np.percentile(rand['V'], 1.))

In [ ]:
rand['RAND_N8'].data.min(), rand['RAND_N8'].data.max()

In [ ]:
_ = pl.hist(rand['RAND_N8'], bins=np.arange(0, 3500, 1), histtype='step')

pl.axvline(rand.meta['NRAND8'], c='k', lw=0.5)

pl.xlabel(r'$N_8$')
pl.xlim(50., 500.)

In [ ]:
# 100. * np.sqrt(rand.meta['NRAND8']) / rand.meta['NRAND8']

In [ ]:
# isin = rand['IS_BOUNDARY'].astype(bool)

pl.scatter(pos_bound[:,0], pos_bound[:,1], lw=0.0, c=bound['V'], s=2)
pl.xlabel('$x$')
pl.ylabel('$y$')
pl.colorbar(label='V')

In [ ]:
pl.scatter(pos[:,0], pos[:,1], lw=0.0, c=rand['BOUND_DIST'], s=2, vmin=0., vmax=70.)
pl.xlabel('$x$')
pl.ylabel('$y$')
pl.colorbar(label='BOUNDARY DISTANCE')

In [ ]:
pl.scatter(pos[:,1], pos[:,2], lw=0.0, c=rand['BOUND_DIST'], s=2, vmin=0., vmax=70.)
pl.xlabel('$y$')
pl.ylabel('$z$')
pl.colorbar(label='BOUNDARY DISTANCE')

In [ ]:
# pl.scatter(pos_bound[:,0], pos_bound[:,1], lw=0.0, c=bound['RAND_N8'], vmin=0, vmax=1500, s=2)
# pl.xlabel('$x$')
# pl.ylabel('$y$')
# pl.colorbar(label=r'$N_8$')

In [ ]:
def theory_ff(bound_dist, radius):
    d = bound_dist
    r = radius
    sphere_cap_vol = 1. - (3. * 8. - d) * d**2 / 4. / 8**3. # [V sphere]
    
    return sphere_cap_vol

d = np.arange(0., 16., 0.1)
r = np.ones_like(d) * 8 # Mpc/h

sphere_cap_vol = 1. - (3. * 8. - d) * d**2 / 4. / 8**3. # [V sphere]

#isin = ~rand['IS_BOUNDARY'].astype(bool)

pl.scatter(rand['BOUND_DIST'], rand['FILLFACTOR'], marker='.', lw=0.0, s=2, alpha=0.1, c='k')

# pl.plot(8. - d, sphere_cap_vol)
pl.plot(8. - d, theory_ff(d, 8), lw=1.)

pl.xlabel('BOUNDARY DISTANCE [$h^{-1}$Mpc]')
pl.ylabel(r'Random $N_8 \ / \ \langle N_8 \rangle$')

pl.axhline(1.00, c='k', lw=0.5)
pl.axvline(8.00, c='k', lw=0.5)

pl.ylim(0.2, 1.1)

d = np.arange(0., 16., 0.1)
r = np.ones_like(d) * 8 # Mpc/h
sphere_cap_vol = 1. - (3. * 8. - d) * d**2 / 4. / 8**3. # [V sphere]

#isin = ~rand['IS_BOUNDARY'].astype(bool)

pl.scatter(rand['BOUND_DIST'], rand['FILLFACTOR'], marker='.', lw=0.0, s=1, alpha=0.1, c='k')
pl.plot(radius - d, sphere_cap_vol)
pl.plot(radius - d, fillfactor_expectation(d, nbar, radius))


pl.xlabel('BOUNDARY DISTANCE [$h^{-1}$Mpc]')
pl.ylabel(r'Random $N_8 \ / \ \langle N_8 \rangle$')

pl.axhline(1.00, c='k', lw=0.5)
pl.axvline(8.00, c='k', lw=0.5)

pl.ylim(0.2, 1.2)

In [ ]:
_ = pl.hist(rand['BOUND_DIST'], bins=50, histtype='step')
pl.xlabel('BOUND_DIST [$h^{-1}$Mpc]')

In [ ]:
fpath = findfile(ftype='ddp_n8', dryrun=dryrun, survey=survey, version=version)
gold  = Table.read(fpath)
gold.meta

In [ ]:
ddp_zmin  = gold.meta['DDP1_ZMIN']
ddp_zmax  = gold.meta['DDP1_ZMAX']

isin      = (rand['Z'] > ddp_zmin) & (rand['Z'] < ddp_zmax)

fills     = rand['FILLFACTOR'].data[isin]
pos_slice = pos[isin]

fills     = fills[np.abs(pos_slice[:,2]) < 5.]
pos_slice = pos_slice[np.abs(pos_slice[:,2]) < 5.]
    
plt.scatter(pos_slice[:,0],\
            pos_slice[:,1],\
            c=fills,\
            marker='.',\
            lw=0.0,\
            s=4,\
            vmin=0.,\
            vmax=1.,\
            cmap='hot')

pl.xlabel('$x$ [$h^{-1}$Mpc]')
pl.ylabel('$y$ [$h^{-1}$Mpc]')

pl.colorbar(label='FILLFACTOR')
    
plt.title('{}'.format(field))
plt.show()

In [ ]:
for idx in np.arange(1, 4, 1):
    ddp_zmin  = gold.meta['DDP{}_ZMIN'.format(idx)]
    ddp_zmax  = gold.meta['DDP{}_ZMAX'.format(idx)]

    isin      = (rand['Z'] > ddp_zmin) & (rand['Z'] < ddp_zmax)

    fills     = rand['DDP{}_DELTA8'.format(idx)].data[isin]
    pos_slice = pos[isin]

    fills     = fills[np.abs(pos_slice[:,2]) < 5.]
    pos_slice = pos_slice[np.abs(pos_slice[:,2]) < 5.]
    
    plt.scatter(pos_slice[:,0],\
                pos_slice[:,1],\
                c=fills,\
                marker='.',\
                lw=0.0,\
                s=1,\
                vmin=0.,\
                vmax=1.,\
                cmap='hot')
    
    pl.xlabel('$x$ [$h^{-1}$Mpc]')
    pl.ylabel('$y$ [$h^{-1}$Mpc]')

    pl.xlim(100.,  800.)
    pl.ylim(-100., 100.)
    
    pl.colorbar(label='DDP{}  $\delta_8$'.format(idx))
    
    plt.title('{}:  DDP{}'.format(field, idx))
    plt.show()

In [ ]:
cmap     = plt.cm.tab20c

cmaplist = [cmap(i) for i in range(cmap.N)]

cmap     = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)

bounds   = np.linspace(0, 9, 10)
norm     = mpl.colors.BoundaryNorm(bounds, cmap.N)

In [ ]:
for idx in np.arange(1, 2, 1):
    ddp_zmin  = gold.meta['DDP{}_ZMIN'.format(idx)]
    ddp_zmax  = gold.meta['DDP{}_ZMAX'.format(idx)]

    isin      = (rand['Z'] > ddp_zmin) & (rand['Z'] < ddp_zmax)
    isin      = rand['DDP{}_DELTA8_TIER'.format(idx)] > -999
    
    fills     = rand['DDP{}_DELTA8_TIER'.format(idx)].data[isin]
    pos_slice = pos[isin]

    fills     = fills[np.abs(pos_slice[:,2]) < 5.]
    pos_slice = pos_slice[np.abs(pos_slice[:,2]) < 5.]
    
    plt.scatter(pos_slice[:,0],\
                pos_slice[:,1],\
                c=fills,\
                marker='.',\
                lw=0.0,\
                s=2,\
                cmap=cmap,\
                norm=norm)
    
    pl.xlabel('$x$ [$h^{-1}$Mpc]')
    pl.ylabel('$y$ [$h^{-1}$Mpc]')

    pl.xlim(100.,  800.)
    pl.ylim(-100., 100.)
    
    pl.colorbar(label='DDP{}  $\delta_8$'.format(idx))
    
    plt.title('{}:  DDP{}'.format(field, idx))

In [ ]:
# Larger minimum redshift & Smaller maximum redshift. 
isin = (rand['Z'] > gold.meta['DDP1_ZMIN']) & (rand['Z'] < gold.meta['DDP2_ZMAX'])

pl.plot(rand['DDP1_DELTA8'][isin], rand['DDP2_DELTA8'][isin], marker=',', lw=0.0, c='k')

pl.xlim(-1.2, 10.)
pl.ylim(-1.2, 10.)

pl.xlabel('DDP1 $\delta_8$')
pl.ylabel('DDP2 $\delta_8$')

In [ ]:
from linlogplot import linlogplot

In [ ]:
linlogplot(rand['DDP1_DELTA8'][isin], rand['DDP2_DELTA8'][isin])

In [ ]:
rand_ddp1fill = np.clip(np.sort(rand[rand['IN_DDP1']]['FILLFACTOR'].data)[::-1], 0, 1) 
rand_ddp2fill = np.clip(np.sort(rand[rand['IN_DDP2']]['FILLFACTOR'].data)[::-1], 0, 1)
rand_ddp3fill = np.clip(np.sort(rand[rand['IN_DDP3']]['FILLFACTOR'].data)[::-1], 0, 1)

In [ ]:
plt.plot(rand_ddp1fill, np.cumsum(rand_ddp1fill) / len(rand_ddp1fill), lw=1, color='blue',  label=r'${:.3f} \leq z \leq {:.3f}$'.format(rand.meta['DDP1_ZMIN'], rand.meta['DDP1_ZMAX']))
plt.plot(rand_ddp2fill, np.cumsum(rand_ddp2fill) / len(rand_ddp2fill), lw=1, color='green', label=r'${:.3f} \leq z \leq {:.3f}$'.format(rand.meta['DDP2_ZMIN'], rand.meta['DDP2_ZMAX']))
plt.plot(rand_ddp3fill, np.cumsum(rand_ddp3fill) / len(rand_ddp3fill), lw=1, color='red',   label=r'${:.3f} \leq z \leq {:.3f}$'.format(rand.meta['DDP3_ZMIN'], rand.meta['DDP3_ZMAX']))

plt.xlabel('Fill factor')
plt.ylabel(r'Volume fraction ($\geq$ Fill factor)')

pl.xlim(0.0, 1.05)
pl.ylim(0.0, 1.05)

plt.legend(frameon=False, loc=1)

# Done.